# DATA INJESTION PIPELINE

In [1]:
from langchain_core.documents import Document

In [2]:
doc= Document(
    page_content= "This is page content to create RAG",
    metadata ={
        "sourcce": "example.txt",
        "pages": 1,
        "author":" me",
        "date created": "2026-01-08"
    }
)
doc

Document(metadata={'sourcce': 'example.txt', 'pages': 1, 'author': ' me', 'date created': '2026-01-08'}, page_content='This is page content to create RAG')

Create a simple txt file

In [3]:
import os
os.makedirs("../data/text_files", exist_ok= True)

sample_texts={

   "../data/text_files/python_intro.txt": """---------Python Introduction-------


Python is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991, Python emphasizes clean syntax and code readability, making it one of the most beginner-friendly languages while still being powerful enough for advanced applications.

One of Python's biggest strengths is its rich ecosystem of libraries and frameworks. It is widely used in data science, machine learning, artificial intelligence, web development, automation, scientific computing, and software testing. Popular libraries such as NumPy, Pandas, TensorFlow, PyTorch, OpenCV, and Matplotlib make Python a top choice for AI and data-related projects, while frameworks like Django and Flask are used for backend web development.

Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It is also platform-independent, meaning the same Python code can run on Windows, macOS, and Linux with minimal changes.

Because of its large community support, extensive documentation, and rapid development capabilities, Python is widely used in academia, research, startups, and industry. Overall, Python is a powerful and flexible programming language that enables faster development and easier problem-solving across a wide range of domains.""",

    "../data/text_files/ml_intro.txt":

 """Machine Learning (ML) is a branch of artificial intelligence that enables systems to learn patterns from data and make predictions or decisions without being explicitly programmed. It focuses on building algorithms that improve performance automatically through experience.

Machine learning is commonly categorized into supervised learning, unsupervised learning, and reinforcement learning. It is widely used in applications such as image and speech recognition, recommendation systems, fraud detection, medical diagnosis, and natural language processing.

Using techniques like regression, classification, clustering, and deep learning, machine learning helps extract meaningful insights from large datasets and supports data-driven decision making. """
}

for filepath, content in sample_texts.items():
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
print("Sample file created!")

Sample file created!


REad the file using text loader

In [4]:
#from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader= TextLoader("../data/text_files/python_intro.txt", encoding = "utf-8")
document= loader.load()
print(document)

/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content="---------Python Introduction-------\n\n\nPython is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991, Python emphasizes clean syntax and code readability, making it one of the most beginner-friendly languages while still being powerful enough for advanced applications.\n\nOne of Python's biggest strengths is its rich ecosystem of libraries and frameworks. It is widely used in data science, machine learning, artificial intelligence, web development, automation, scientific computing, and software testing. Popular libraries such as NumPy, Pandas, TensorFlow, PyTorch, OpenCV, and Matplotlib make Python a top choice for AI and data-related projects, while frameworks like Django and Flask are used for backend web development.\n\nPython supports multiple programming paradigms, including procedural, objec

REad file using directory loader

In [5]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader= DirectoryLoader(
    "../data/text_files", 
    glob= "**/*.txt", #Pattern to match
    loader_cls= TextLoader,
    loader_kwargs= {'encoding': 'utf-8'},
    show_progress= False
)
dir_doc= dir_loader.load()
print(dir_doc)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content="---------Python Introduction-------\n\n\nPython is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991, Python emphasizes clean syntax and code readability, making it one of the most beginner-friendly languages while still being powerful enough for advanced applications.\n\nOne of Python's biggest strengths is its rich ecosystem of libraries and frameworks. It is widely used in data science, machine learning, artificial intelligence, web development, automation, scientific computing, and software testing. Popular libraries such as NumPy, Pandas, TensorFlow, PyTorch, OpenCV, and Matplotlib make Python a top choice for AI and data-related projects, while frameworks like Django and Flask are used for backend web development.\n\nPython supports multiple programming paradigms, including procedural, objec

load pdf files

In [6]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

pdf_loader= DirectoryLoader(
    "../data/pdf", 
    glob= "**/*.pdf", #Pattern to match
    loader_cls= PyMuPDFLoader,
    show_progress= False
)
pdf_doc= pdf_loader.load()
print(pdf_doc)

print(f"Total pages loaded: {len(pdf_doc)}")
print("First page preview:")
print(pdf_doc[0].page_content[:500])

[Document(metadata={'producer': 'PyPDF2', 'creator': '', 'creationdate': '', 'source': '../data/pdf/attention-is-all-you-need.pdf', 'file_path': '../data/pdf/attention-is-all-you-need.pdf', 'total_pages': 11, 'format': 'PDF 1.3', 'title': 'Attention is All you Need', 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'keywords': '', 'moddate': '2018-02-12T21:22:10-08:00', 'trapped': '', 'modDate': "D:20180212212210-08'00'", 'creationDate': '', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszka

In [7]:
type(pdf_doc[0])

langchain_core.documents.base.Document

Embedding and VectorSrore DB

In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
class EmbeddingManager:
    """Handels embeddings usinf SentenceTransformer"""
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """Initialize embedding manager
        model_name: available in HuggingFace
        """
        self.model_name= model_name
        self.model= None
        self._load_model()

    def _load_model(self):
        """Loads the model"""
        try:
            print("Loading model...")
            self.model= SentenceTransformer(self.model_name)
            print(f"Model Loaded! Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print("Error loading the model")
            raise

    def generate_embeddings(self, texts: List[str])-> np.ndarray:
        """Generate embeddings for the lsit of texts"""
        if not self.model:
            raise ValueError("MOdel not loaded")
        print("Generating embeddings...")
        embeddings= self.model.encode(texts, show_progress_bar= True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

    def get_sentence_embedding_dimension(self) -> int:
        """Get the embedding dimension for the mdoel"""
        if not self.model:
            raise ValueError("MOdel not loaded")
        return seld.model.get_sentence_embedding_dimension()


#Initialize the embedding manager
embedding_manager= EmbeddingManager()
embedding_manager

Loading model...
Model Loaded! Embedding dimension: 384


Vector Store

In [10]:
class VectorStore:
    """Manages document embeddings in ChromaDB"""
    
    def __init__(self, collection_name: str = "pdf-documents", persist_directory: str = "../data/vector_store"):
        """Initialize vector store
        collection_name: Name of ChromaDB collection
        persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self.initialize_store()

    def initialize_store(self):
        """Initialize the ChromaDB client and collection"""
        try:
            os.makedirs(self.persist_directory, exist_ok=True)
            
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={
                    "description": "PDF document embeddings for RAG"
                }
            )
            
            print(f"Vector Store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print("Error initializing the store")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """Add documents and their embeddings to vector store
        documents: List of LangChain Document objects
        embeddings: Corresponding embeddings
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match the number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['context_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error storing to vector store: {e}")
            raise


# Initialize vector store
vector_store = VectorStore()
vector_store

Vector Store initialized. Collection: pdf-documents
Existing documents in collection: 0


In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """
    Split documents into smaller chunks for better RAG performance.
    
    Parameters:
    - chunk_size: Maximum characters per chunk (adjust based on your LLM)
    - chunk_overlap: Characters to overlap between chunks (preserves context)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, # Each chunk: ~1000 characters
        chunk_overlap=chunk_overlap, # 200 chars overlap for context
        length_function=len, # How to measure length
        separators=["\n\n", "\n", " ", ""] # Split hierarchy
    )
    # Actually split the documents
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show what a chunk looks like
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [12]:
chunks= split_documents(pdf_doc)
chunks

Split 15 documents into 59 chunks

Example chunk:
Content: Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz...
Metadata: {'producer': 'PyPDF2', 'creator': '', 'creationdate': '', 'source': '../data/pdf/attention-is-all-you-need.pdf', 'file_path': '../data/pdf/attention-is-all-you-need.pdf', 'total_pages': 11, 'format': 'PDF 1.3', 'title': 'Attention is All you Need', 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'keywords': '', 'moddate': '2018-02-12T21:22:10-08:00', 'trapped': '', 'modDate': "D:20180212212210-08'00'", 'creationDate': '', 'page': 0}


[Document(metadata={'producer': 'PyPDF2', 'creator': '', 'creationdate': '', 'source': '../data/pdf/attention-is-all-you-need.pdf', 'file_path': '../data/pdf/attention-is-all-you-need.pdf', 'total_pages': 11, 'format': 'PDF 1.3', 'title': 'Attention is All you Need', 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'keywords': '', 'moddate': '2018-02-12T21:22:10-08:00', 'trapped': '', 'modDate': "D:20180212212210-08'00'", 'creationDate': '', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszka

COnvert the text to embeddings

In [13]:
texts= [doc.page_content for doc in chunks]
texts

['Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantl

In [14]:
texts= [doc.page_content for doc in chunks]

#Generate the embeddings
embeddings= embedding_manager.generate_embeddings(texts)

#store it in vecto DB
vector_store.add_documents(chunks, embeddings)

Generating embeddings...


Batches: 100%|██████████| 2/2 [00:00<00:00,  8.23it/s]

Generated embeddings with shape: (59, 384)
Adding 59 documents to vector store...
Successfully added 59 documents to vector store
Total documents in collection: 59


# RETRIVAL PIPELINE

In [15]:
class RAGRetriever:
    """Handles query-based retrieval from vector_store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """Initialize the retriever
        vector_store: Vector store from ChromaDB
        embedding_manager: Manager for generating embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """Retrieve relevant documents for a query
        query: Search query
        top_k: Number of top results to return
        score_threshold: Minimum similarity score threshold
        
        returns: list of dicts containing retrieved documents and metadata
        """
        print(f"Retrieving for query: '{query}'")
        print(f"Top k: {top_k}, score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")

            else:
                print("No documents found!")

            return retrieved_docs

        except Exception as e:
            print(f"Error retrieving!: {e}")
            return []

rag_retriever= RAGRetriever(vector_store, embedding_manager)

In [16]:
rag_retriever

In [18]:
rag_retriever.retrieve("What is attetion is all you need?")

Retrieving for query: 'What is attetion is all you need?'
Top k: 5, score threshold: 0.0
Generating embeddings...


Batches: 100%|██████████| 1/1 [00:00<00:00, 31.46it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]